# IVR7 Intermediate 2 Example
This example aims to highlight the follow:
* Builds a simple pickup/dropoff (similar to the basic model).
* Runs an eval on a sub-sequence to illustrate how to call the endpoint (with an evaluate sequence) and
* Interpret the responses in terms of infeasibility messages.

## Requirements
This example assumes you've configured an api-key with the required services enabled (see the [portal](portal.icepack.ai) for configuration details) and that you're familiar with loading and working with protobuf models.

## Data
We're going to load sample data which has order sizes and durations in a tabular format.


In [ ]:
import pandas
df = pandas.read_csv('../sample_data/publist_orders.csv')
print(df.head())

## Model Configuration
This model builds the ivr7-basic example.
The same dimensions (distance,time and capacity) are used as well as a handful of jobs, one vehicle cost class, one vehicle class and five vehicles.

In [ ]:
exec(open('apiHelper.py').read()) # import some api-helper classes which we've written for you.
exec(open('ivr7-model-helper.py').read()) # import some modelling helpers


api = apiHelper(modelType="ivr7-kt461v8eoaif") # set the model type to the ivr7 model

sr = ivr7_kt461v8eoaif_pb2.SolveRequest()
sr.solveType = 0 # optimise solve request

m = sr.model
m.dimensions.CopyFrom(make_distance_time_cap_dims())
m.locations.extend(make_locations(df)) #using the wrapper function
m.jobs.extend(make_job_time_cap(df, [0] * (df.shape[0]-1), range(1, df.shape[0])))
m.vehicleCostClasses.append(make_vcc_simple('vcc1', 1000, 1.001e-2, 1.001e-2, 1.001e-2, 1, 3))
m.vehicleClasses.append(make_vc_simple('vc1', 1, 1, 1, 1))
for i in range(1,5):
    m.vehicles.append( make_vehicle_cap('vehicle_' + str(i), 'vc1', 'vcc1',
                                        2000, # the vehicle capacity
                                        df['id'][0], # the start location
                                        df['id'][0], # the end location
                                        7*60,        # the start time (7AM)
                                        18*60))       # the end time  (6PM)
# at this point we have a complete model which we can run. So we can go ahead and do that!

reqId = api.Post(sr)
sol = api.Get(reqId)
t = tabulate(sr, sol)

t['nodes'].head()



### Adding constraints to the model
There are a few things we could do to the model now. We have a valid response for the model we ran - but we can update the model with additional constraints and then **evaluate** the existing solution to see which constraints are broken given the previous solution

In [ ]:
# lets add some time windows to the model.
for i,e in enumerate(m.locations):
    la = ivr7_kt461v8eoaif_pb2.Location.Attribute()
    la.dimensionId = 'time'
    la.quantity = 0
    w = ivr7_kt461v8eoaif_pb2.Window()
    w.start = 8*60
    w.end = 14*60
    la.arrivalWindows.append(w)
    m.locations[i].attributes.append(la)

print(m.locations)

### Extracting the task-sequence
The aim now is to add a task-sequence to the model. In order to do that, we need to organise the data according to vehicle (i.e. provide a task-sequence per vehicle). We can query the tabulated data for the sequence filtering out the vehicle-start and vehicle-end nodes (these are implicitly created in the model).

In [ ]:
nodes = t['nodes']
del m.taskSequence[:]
for vid in set(nodes.vehicleId):
    tseq = ivr7_kt461v8eoaif_pb2.TaskSequence()
    tseq.vehicleId = vid
    tseq.taskId.extend(list(nodes[~nodes.taskId.str.contains('Shift') & (nodes['vehicleId']==vid)].taskId))
    m.taskSequence.append(tseq)
    
print(m.taskSequence) # this then defines the sequence for each vehicle which we want to evalute

### Re-run the model
We're going to submit this modified model to the api - but this time we're going to save the results separately so that we can contrast it against the model which has already been run.\

**Rememeber:** we need to change the solve request type to an evaluate request - otherwise the solver will simply solve this new problem within the constraints specified, when what we want is to actually see which constraints are broken.

In [ ]:
sr.solveType = 1 # for an evaluate request
reqId = api.Post(sr)
evalsol = api.Get(reqId)
evalt = tabulate(sr, evalsol)

Lets check the solution response and see if any infeasibilities were generated as a result of the additional constraints added to the model.

In [ ]:
print(evalsol.infeasibilities)
# so there are quite a few here - which is okay. Lets check it in tabular form.

In [ ]:
evalt['infeasibilities']

So here we should have some constraints which have been broken.
We get told which dimension is related (if the constraint is related to a dimension) as well as which type of constraint (if known) and the degree to which the constraint is broken.

If the constraints are tardy constraints being broken, this means the task starts AFTER the allowable window. The limit will be often be zero, the value will be the amount by which the vehicle is late. We can check the arrival time of the task to verify this:

In [ ]:
nodes = evalt['nodes']
infeasibleTasks = evalt['infeasibilities'].taskId

latestops = nodes[nodes.taskId.isin(infeasibleTasks)][['stopId', 'sequence', 'locationId', 'taskId', 'time_start', 'time_end']]
latestops['windowBroken'] = latestops['time_start'] > 14*60
latestops


This verifies that all these tasks flagged as infeasible are indeed breaking the window constraint we added to the original model. These kinds of exercises are useful if you perform a drag-and-drop on an interface to move stops around and want to see if the resulting route is feasible or not (and perhaps indicate to a user which tasks are infeasible given the proposed sequence).

We can simply remove the tasks from the evaluation sequence which are breaking constraints. You'll notice though, that most of the tasks breaking the constraints are actually the dropoff tasks (because the pickups occur within the feasible window of time) 

In [ ]:
for i, e in enumerate(m.taskSequence):
    remainingTasks = list()
    for ti, tsk in enumerate(e.taskId):
        if tsk not in list(latestops.taskId):
            remainingTasks.append(tsk)
    del e.taskId[:] 
    e.taskId.extend(remainingTasks)


We can then go ahead and rerun the the model without the dropoff tasks (which have been excluded from the task-sequence above)

In [ ]:
reqId = api.Post(sr)
evalsol = api.Get(reqId)
evalt = tabulate(sr, evalsol)
evalt['infeasibilities']

This is again, rather intuitive. We find that there are a whole bunch of precendence constraints which are then broken, cumul-pair constraints and task-pair constraints. This is because there's a relation between the pickup and dropoff and either they're BOTH scheduled or BOTH unscheduled. Having one task assigned to a vehicle in the schedule without the other breaks a bunch of constraints. So lets apply the same trick and remove these pickup stops which are breaking constraints.

In [ ]:
badTasks = list()
for i, e in enumerate(evalsol.infeasibilities):
    badTasks.append(e.taskId)

for i, e in enumerate(m.taskSequence):
    remainingTasks = list()
    for ti, tsk in enumerate(e.taskId):
        if tsk not in badTasks:
            remainingTasks.append(tsk)
    del e.taskId[:] 
    e.taskId.extend(remainingTasks)
# and then run the model again
reqId = api.Post(sr)
evalsol = api.Get(reqId)
evalt = tabulate(sr, evalsol)
evalt['infeasibilities']

and now there are no infeasibilities returned by the solver. So the remaining stops that are assigned to the vehicles are indeed feasible. The catch here is that the solution cost is quite a bit higher than if we had simply let the scheduler re-work the schedule around the constraints that we added.


In [ ]:
print(sol.objective)
print(evalsol.objective)
print(sol.objective < evalsol.objective) # this is because we now have orders which have been omitted from the schedule

We can simply modify the solve request which is set to `evaluate` back to `optimise` in order to see if a better solution can be found within the new constraints.

In [ ]:
sr.solveType = 0
reqId = api.Post(sr)
windowsol = api.Get(reqId)
t_windows = tabulate(sr, windowsol)
print(t_windows['infeasibilities'])
print(windowsol.objective)

The solution found by the solver does indeed cost more than before, but, there are no broken constraints and it found a solution which meets all the demand, making it cheaper than the evaluation sequences we provided.
I.e:

In [ ]:
print(windowsol.objective < evalsol.objective)
print(sol.objective < windowsol.objective)

### What next?
The advanced ivr examples are a good next step if you're comfortable with this example.